In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/1.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')

postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,akjmXXq,0,FEM,28.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,MV6b6R9,0,FEM,30.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,zvxv1Rm,0,FEM,22.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,8oqJzL,0,MASC,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ez533B,0,MASC,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

12377268
12332167


In [9]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
10729878,1112298005,EzELAZ9,0,FEM,19.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,True,False,NaN
1867564,1111934356,ekmlqbj,1,FEM,29.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,True,True,False,True,NaN
1955676,1112285529,ek4wxbo,1,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
2335273,1112253336,8MZVxpL,1,MASC,28.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
2293586,1112309001,kPNE41a,1,FEM,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,NaN
10581954,1112452752,3NkBmdr,0,FEM,25.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
9370877,1112402823,54zDMp,0,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
5057661,1112218941,wVXPeZL,1,MASC,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
10802022,1112222991,5PvpGG,0,MASC,38.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,True,False,True,NaN
8345219,1112309025,MVdXkkN,0,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
2931355,1112439246,YGpNkr,1,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
4366376,1112399864,xkve6vP,1,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,True,False,True,0.0
5222944,1112285761,vVjGva5,1,MASC,19.0,-,-,-,-,Graduado,...,False,False,True,False,True,True,False,False,False,0.0
3309171,1112273328,PmGKoO0,1,MASC,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,0.0
12205117,1112444563,YAZ0w1,0,MASC,49.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
2176587,1112335018,5DmMrZ,1,FEM,32.0,-,-,En Curso,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
2920414,1112334788,Nz01w05,1,FEM,21.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.0
1884035,1112282919,NxBNrl,1,FEM,44.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,False,0.0
6191451,1112235610,KBrVWj2,0,MASC,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
7481755,1111753928,52OpQp,0,FEM,41.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0


In [11]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
postulaciones2=0

Train:  11098950 Test:  1233217


In [12]:
import gc
gc.collect()

379

In [13]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1000,colsample_bytree=0.7,n_jobs=-1,random_state=0,silent=False,subsample=0.7,sumsample_freq=1)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [14]:
score=c.score(x_test,y_test)*100
print(score)

73.12541101849878


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV4.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV4.pkl']

In [16]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
80109,27594,1112352275,5GewEX,FEM,28.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
42052,48662,1112433770,MVOO20v,FEM,25.0,-,-,Graduado,-,Graduado,...,False,False,False,False,True,True,True,False,True,1.0
95750,95234,1112487995,aXGmKq,MASC,36.0,-,En Curso,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
20121,81062,1112465355,BmDBVbz,MASC,30.0,-,-,-,-,-,...,False,False,True,False,False,True,False,False,False,2.0
64722,11654,1111963981,ZBDVYz,MASC,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
81836,29683,1112373339,axG1G9,MASC,49.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
39443,83789,1112466410,96zMzeZ,FEM,20.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
67391,13477,1112040532,Y6PD1,MASC,42.0,-,Graduado,-,Graduado,Graduado,...,False,False,False,False,True,True,False,True,False,0.0
63678,53888,1112445541,YjlbJzr,FEM,23.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
24188,79205,1112464639,1PjLab,FEM,35.0,-,-,-,-,-,...,False,True,False,False,False,False,False,False,False,0.0


In [17]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.57340882, 0.42659118],
       [0.60312152, 0.39687848],
       [0.61200069, 0.38799931],
       ...,
       [0.87191205, 0.12808795],
       [0.63907433, 0.36092567],
       [0.95238621, 0.04761379]])

In [18]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [19]:
prediccion['sepostulo']=y_final2

In [20]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.426591
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.396878
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.387999
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.231194
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.320712


In [21]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.426591
1,1,0.396878
2,2,0.387999
3,3,0.231194
4,4,0.320712


In [22]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion17.csv",index=False)

In [23]:
prediccion.sepostulo.mean()

0.5987484259566261